### 使用 Unsloth 框架对 Qwen2.5-7B 模型进行微调的示例代码
### 本代码可以在免费的 Tesla T4 Google Colab 实例上运行 https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen2.5_(7B)-Alpaca.ipynb

In [1]:
# 导入必要的库
from unsloth import FastLanguageModel
import torch

# 设置模型参数
max_seq_length = 2048  # 设置最大序列长度，支持 RoPE 缩放
dtype = None  # 数据类型，None 表示自动检测。Tesla T4 使用 Float16，Ampere+ 使用 Bfloat16
load_in_4bit = True  # 使用 4bit 量化来减少内存使用

# 加载预训练模型和分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/root/autodl-tmp/models/Qwen/Qwen2___5-7B-Instruct",  # 使用Qwen2.5-7B模型
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.6: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.546 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
# 添加LoRA适配器，只需要更新1-10%的参数
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # LoRA秩，建议使用8、16、32、64、128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],  # 需要应用LoRA的模块
    lora_alpha = 16,  # LoRA缩放因子
    lora_dropout = 0,  # LoRA dropout率，0为优化设置
    bias = "none",    # 偏置项设置，none为优化设置
    use_gradient_checkpointing = "unsloth",  # 使用unsloth的梯度检查点，可减少30%显存使用
    random_state = 3407,  # 随机种子
    use_rslora = False,  # 是否使用rank stabilized LoRA
    loftq_config = None,  # LoftQ配置
)

### 数据准备

In [3]:
import os
import pandas as pd
from datasets import Dataset

# 定义医疗对话的提示模板
medical_prompt = """你是一个专业的医疗助手。请根据患者的问题提供专业、准确的回答。

### 问题：
{}

### 回答：
{}"""

# 获取结束标记
EOS_TOKEN = tokenizer.eos_token

def read_csv_with_encoding(file_path):
    """尝试使用不同的编码读取CSV文件"""
    encodings = ['gbk', 'gb2312', 'gb18030', 'utf-8']
    for encoding in encodings:
        try:
            return pd.read_csv(file_path, encoding=encoding)
        except UnicodeDecodeError:
            continue
    raise ValueError(f"无法使用任何编码读取文件: {file_path}")

def load_medical_data(data_dir):
    """加载医疗对话数据"""
    data = []
    departments = {
        'IM_内科': '内科',
        'Surgical_外科': '外科',
        'Pediatric_儿科': '儿科',
        'Oncology_肿瘤科': '肿瘤科',
        'OAGD_妇产科': '妇产科',
        'Andriatria_男科': '男科'
    }
    
    # 遍历所有科室目录
    for dept_dir, dept_name in departments.items():
        dept_path = os.path.join(data_dir, dept_dir)
        if not os.path.exists(dept_path):
            print(f"目录不存在: {dept_path}")
            continue
            
        print(f"\n处理{dept_name}数据...")
        
        # 获取该科室下的所有CSV文件
        csv_files = [f for f in os.listdir(dept_path) if f.endswith('.csv')]
        
        for csv_file in csv_files:
            file_path = os.path.join(dept_path, csv_file)
            print(f"正在处理文件: {csv_file}")
            
            try:
                # 读取CSV文件
                df = read_csv_with_encoding(file_path)
                
                # 打印列名，帮助调试
                print(f"文件 {csv_file} 的列名: {df.columns.tolist()}")
                
                # 处理每一行数据
                for _, row in df.iterrows():
                    try:
                        # 获取问题和回答（尝试不同的列名）
                        question = None
                        answer = None
                        
                        # 尝试不同的列名
                        if 'question' in row:
                            question = str(row['question']).strip()
                        elif '问题' in row:
                            question = str(row['问题']).strip()
                        elif 'ask' in row:
                            question = str(row['ask']).strip()
                            
                        if 'answer' in row:
                            answer = str(row['answer']).strip()
                        elif '回答' in row:
                            answer = str(row['回答']).strip()
                        elif 'response' in row:
                            answer = str(row['response']).strip()
                        
                        # 过滤无效数据
                        if not question or not answer:
                            continue
                            
                        # 限制长度
                        if len(question) > 200 or len(answer) > 200:
                            continue
                            
                        # 添加到数据列表
                        data.append({
                            "instruction": "请回答以下医疗相关问题",
                            "input": question,
                            "output": answer
                        })
                        
                    except Exception as e:
                        print(f"处理数据行时出错: {e}")
                        continue
                        
            except Exception as e:
                print(f"处理文件 {csv_file} 时出错: {e}")
                continue
    
    # 验证数据
    if not data:
        raise ValueError("没有成功处理任何数据！")
        
    print(f"\n成功处理 {len(data)} 条数据")
    return Dataset.from_list(data)

def formatting_prompts_func(examples):
    """格式化提示"""
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = medical_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# 加载医疗数据集
dataset = load_medical_data("【数据集】中文医疗数据")
dataset = dataset.map(formatting_prompts_func, batched=True)


处理内科数据...
正在处理文件: 内科5000-33000.csv
文件 内科5000-33000.csv 的列名: ['department', 'title', 'ask', 'answer']

处理外科数据...
正在处理文件: 外科5-14000.csv
文件 外科5-14000.csv 的列名: ['department', 'title', 'ask', 'answer']

处理儿科数据...
正在处理文件: 儿科5-14000.csv
文件 儿科5-14000.csv 的列名: ['department', 'title', 'ask', 'answer']

处理肿瘤科数据...
正在处理文件: 肿瘤科5-10000.csv
文件 肿瘤科5-10000.csv 的列名: ['department', 'title', 'ask', 'answer']

处理妇产科数据...
正在处理文件: 妇产科6-28000.csv
文件 妇产科6-28000.csv 的列名: ['department', 'title', 'ask', 'answer']

处理男科数据...
正在处理文件: 男科5-13000.csv
文件 男科5-13000.csv 的列名: ['department', 'title', 'ask', 'answer']

成功处理 664001 条数据


Map:   0%|          | 0/664001 [00:00<?, ? examples/s]

### 模型训练

In [4]:
# 设置训练参数和训练器
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# 定义训练参数:3轮全量训练
# training_args = TrainingArguments(
#         per_device_train_batch_size = 2,  # 每个设备的训练批次大小
#         gradient_accumulation_steps = 4,  # 梯度累积步数
#         warmup_steps = 5,  # 预热步数
#         #max_steps = 60,  # 最大训练步数
#         max_steps = -1,  # 不使用max_steps
#         num_train_epochs = 3,  # 训练3个epoch
#         learning_rate = 2e-4,  # 学习率
#         fp16 = not is_bfloat16_supported(),  # 是否使用FP16
#         bf16 = is_bfloat16_supported(),  # 是否使用BF16
#         logging_steps = 1,  # 日志记录步数
#         optim = "adamw_8bit",  # 优化器
#         weight_decay = 0.01,  # 权重衰减
#         lr_scheduler_type = "linear",  # 学习率调度器类型
#         seed = 3407,  # 随机种子
#         output_dir = "outputs",  # 输出目录
#         report_to = "none",  # 报告方式
#     )
#测试，先训练60步，调通后，再训练3600步试试效果。
training_args = TrainingArguments(
        per_device_train_batch_size = 2,  # 每个设备的训练批次大小
        gradient_accumulation_steps = 4,  # 梯度累积步数
        warmup_steps = 5,  # 预热步数
        max_steps = 3600,  # 最大训练步数
        learning_rate = 2e-4,  # 学习率
        fp16 = not is_bfloat16_supported(),  # 是否使用FP16
        bf16 = is_bfloat16_supported(),  # 是否使用BF16
        logging_steps = 1,  # 日志记录步数
        optim = "adamw_8bit",  # 优化器
        weight_decay = 0.01,  # 权重衰减
        lr_scheduler_type = "linear",  # 学习率调度器类型
        seed = 3407,  # 随机种子
        output_dir = "outputs",  # 输出目录
        report_to = "none",  # 报告方式
    )
# 创建SFTTrainer实例
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,  # 对于短序列可以设置为True，训练速度提升5倍
    args = training_args,
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/664001 [00:00<?, ? examples/s]

In [5]:
# 显示当前GPU内存状态
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.546 GB.
7.213 GB of memory reserved.


In [6]:
# 开始训练
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 664,001 | Num Epochs = 1 | Total steps = 3,600
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176/7,000,000,000 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.367400
2,2.911500
3,3.022900
4,2.949400
5,2.898300
6,2.672000
7,2.284300
8,2.531800
9,2.012400
10,2.118700


In [7]:
# 显示训练后的内存和时间统计
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

4482.6878 seconds used for training.
74.71 minutes used for training.
Peak reserved memory = 7.213 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 30.634 %.
Peak reserved memory for training % of max memory = 0.0 %.


### 模型推理

In [8]:
# # 模型推理示例
# def generate_medical_response(question):
#     """生成医疗回答"""
#     FastLanguageModel.for_inference(model)  # 启用原生2倍速推理
#     inputs = tokenizer(
#         [medical_prompt.format(question, "")],
#         return_tensors="pt"
#     ).to("cuda")
    
#     from transformers import TextStreamer
#     text_streamer = TextStreamer(tokenizer)
#     _ = model.generate(
#         **inputs,
#         streamer=text_streamer,
#         max_new_tokens=256,
#         temperature=0.7,
#         top_p=0.9,
#         repetition_penalty=1.1
#     )
# 生成医疗回答的函数
"""
:function: generate_medical_response
:param question: str，用户输入的问题
:return: None，直接输出模型回答
"""
def generate_medical_response(question):
    """生成医疗回答"""
    FastLanguageModel.for_inference(model)  # 启用原生2倍速推理
    inputs = tokenizer(
        [medical_prompt.format(question, "")],
        return_tensors="pt"
    ).to("cuda")
    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer)
    _ = model.generate(
        **inputs,
        streamer=text_streamer,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1
    )
    


In [9]:
# 测试问题
test_questions = [
    "我最近总是感觉头晕，应该怎么办？",
    "感冒发烧应该吃什么药？",
    "高血压患者需要注意什么？"
]

for question in test_questions:
    print("\n" + "="*50)
    print(f"问题：{question}")
    print("回答：")
    generate_medical_response(question) 


问题：我最近总是感觉头晕，应该怎么办？
回答：
你是一个专业的医疗助手。请根据患者的问题提供专业、准确的回答。

### 问题：
我最近总是感觉头晕，应该怎么办？

### 回答：
你好！引起头晕的原因有很多,比如颈椎病引起的植物神经调节紊乱,美尼尔氏综合征等,建议到医院的内科就诊检查一下.如果排除了这些原因的话就是贫血或者低血压导致的症状,平时注意多吃些补血的食物如红枣,枸杞,阿胶,桂圆,猪肝,瘦肉,蛋黄等等,另外要注意休息不要熬夜.<|im_end|>

问题：感冒发烧应该吃什么药？
回答：
你是一个专业的医疗助手。请根据患者的问题提供专业、准确的回答。

### 问题：
感冒发烧应该吃什么药？

### 回答：
你好，感冒发烧可以服用感康、清开灵等药物治疗的，注意多喝水，清淡饮食，忌烟酒刺激性食物。严重的话到医院输液治疗。，对于呼吸内科疾病疾病的出现，患者朋友们应该做到早发现早治疗，因为早期的呼吸内科疾病是容易得到控制的。患者们不要错过治疗的好时机。<|im_end|>

问题：高血压患者需要注意什么？
回答：
你是一个专业的医疗助手。请根据患者的问题提供专业、准确的回答。

### 问题：
高血压患者需要注意什么？

### 回答：
您好，您所说的高血压是一种以动脉压升高为特征，可伴有心脏、血管、脑和肾脏等器官功能性或器质性改变的全身性疾病，高血压一般跟不良的生活习惯有关，如：高盐饮食、吸烟、饮酒，肥胖等等，另外还有遗传因素。高血压目前是不能根治的，只能说从药物稳定血压，所以要终生服药。
建议您低盐饮食,低脂饮食.一定要禁烟 ,酒,避免情绪激动,肥甘厚味.口服尼群地平片 非洛地平片 ，卡托普利片 血管紧张素转化酶抑制剂。一定要坚持治疗<|im_end|>


### 微调模型保存
**[注意]** 这里只是LoRA参数，不是完整模型。

In [10]:
# 保存模型
model.save_pretrained("lora_model_medical")  # 本地保存
tokenizer.save_pretrained("lora_model_medical")

('lora_model_medical/tokenizer_config.json',
 'lora_model_medical/special_tokens_map.json',
 'lora_model_medical/vocab.json',
 'lora_model_medical/merges.txt',
 'lora_model_medical/added_tokens.json',
 'lora_model_medical/tokenizer.json')

In [11]:
# 加载保存的模型进行推理
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/root/autodl-tmp/models/Qwen/Qwen2___5-7B-Instruct",  # 基础模型
        adapter_name = "lora_model_medical",  # LoRA权重
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)  # 启用原生2倍速推理



==((====))==  Unsloth 2025.5.6: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.546 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
question = "我最近总是感觉头晕，应该怎么办？"
generate_medical_response(question)

你是一个专业的医疗助手。请根据患者的问题提供专业、准确的回答。

### 问题：
我最近总是感觉头晕，应该怎么办？

### 回答：
感到经常性头晕可能由多种原因引起，包括但不限于脱水、低血压、贫血、耳部疾病（如美尼尔氏病）、颈椎问题等。建议您采取以下步骤：

1. **保持良好的生活习惯**：确保充足的睡眠，避免过度劳累；保证饮食均衡，摄入足够的水分和营养。
2. **适量运动**：适度的体育活动可以提高身体素质，但要避免剧烈或长时间运动。
2. **监测症状**：记录头晕发作的时间、频率以及任何可能诱发因素，这将有助于医生进行诊断。
3. **及时就医**：如果头晕情况持续存在或伴有其他严重症状（如视力模糊、言语不清、肢体无力等），应尽快就医进行全面检查。医生可能会要求做一些检查，比如血液检测、眼压测量或是影像学检查来确定具体原因，并给出相应的治疗建议。

请注意，以上信息仅供参考，并不能代替专业医疗服务。如有需要，请咨询相关领域的专业人士以获得个性化的指导和帮助。同时，在未明确病因之前，请不要自行用药以免延误病情。希望您早日康复！#健康 #医疗建议 #头晕处理方法 #医疗知识普及 #预防措施 #生活方式调整 #就医指南 #症状管理 #健康生活小贴士 #


In [13]:
# 10个中医求诊相关的示例问题
example_questions = [
    "我最近总是感觉头晕，应该怎么办？",
    "晚上经常失眠，有什么中医调理的方法？",
    "经常口干口苦，是身体哪里出了问题？",
    "最近食欲不振，容易腹胀，有什么建议？",
    "女性月经不调，中医如何调理？",
    "经常腰膝酸软，是肾虚吗？",
    "小孩经常咳嗽，有什么中医食疗方？",
    "体质虚弱，容易感冒，如何增强体质？",
    "皮肤经常过敏发痒，中医怎么看？",
    "老年人夜尿频多，有什么中医建议？"
]

# 打印示例问题供用户选择
print("请选择或输入您的中医求诊问题（输入'exit'或'退出'结束）：")
for idx, q in enumerate(example_questions, 1):
    print(f"{idx}. {q}")

# 循环询问用户问题，直到输入exit或退出
while True:
    user_input = input("\n请输入您的问题（可输入序号或自定义问题）：\n")
    if user_input.strip().lower() in ["exit", "退出"]:
        print("感谢您的咨询，再见！")
        break
    # 如果输入为数字序号，自动选择示例问题
    if user_input.isdigit():
        idx = int(user_input)
        if 1 <= idx <= len(example_questions):
            question = example_questions[idx-1]
        else:
            print("无效的序号，请重新输入。")
            continue
    else:
        question = user_input
    generate_medical_response(question) 

请选择或输入您的中医求诊问题（输入'exit'或'退出'结束）：
1. 我最近总是感觉头晕，应该怎么办？
2. 晚上经常失眠，有什么中医调理的方法？
3. 经常口干口苦，是身体哪里出了问题？
4. 最近食欲不振，容易腹胀，有什么建议？
5. 女性月经不调，中医如何调理？
6. 经常腰膝酸软，是肾虚吗？
7. 小孩经常咳嗽，有什么中医食疗方？
8. 体质虚弱，容易感冒，如何增强体质？
9. 皮肤经常过敏发痒，中医怎么看？
10. 老年人夜尿频多，有什么中医建议？



请输入您的问题（可输入序号或自定义问题）：
 10


你是一个专业的医疗助手。请根据患者的问题提供专业、准确的回答。

### 问题：
老年人夜尿频多，有什么中医建议？

### 回答：
老年人夜尿频多是临床上较为常见的一种症状，主要与肾功能减退、前列腺疾病等因素有关，从中医角度分析，可能涉及肝肾阴虚、脾肾两虚等证型。以下是一些中医方面的调治建议：

1. **饮食调理**：建议食用一些有助于补肾益气的食物，如黑豆、核桃仁、枸杞子、山药等；避免辛辣刺激性食物，减少盐分摄入，以防加重肾脏负担。

2. **中药调理**：
   - 肝肾阴虚者可选用六味地黄丸或知柏地黄丸。
   - 脾肾两虚者则可用金匮肾气丸或者右归丸进行调理。
   
2. **生活方式调整**：
   - 保持规律的作息时间，避免熬夜，保证充足的睡眠。
   - 适当参加体育锻炼，增强体质，但要避免剧烈运动，以免伤身。
   - 减少夜间饮水量，尤其是睡前几小时内尽量控制液体摄入。
   - 注意保暖，特别是腰部和下肢的保暖措施可以预防寒湿邪气入侵。

3. **针灸疗法**：通过针刺太溪穴（位于脚内



请输入您的问题（可输入序号或自定义问题）：
 6


你是一个专业的医疗助手。请根据患者的问题提供专业、准确的回答。

### 问题：
经常腰膝酸软，是肾虚吗？

### 回答：
腰膝酸软是一种常见的症状，可能与多种因素有关，并不一定是由肾虚引起的。中医认为肾藏精，主生长发育和生殖功能，在人体的生理活动中起着重要作用，因此肾虚是可能导致腰膝酸软的原因之一。但是，现代医学中并没有“肾虚”这个概念，而是将腰膝酸软等症状归因于不同的疾病或病理状态，如慢性疲劳综合征、骨质疏松症、椎间盘突出等。

建议您在出现此类症状时，首先应该到正规医院进行全面检查以明确病因，包括但不限于血液检查、尿液检查、影像学检查（X光片、MRI等）。同时，结合自身的生活习惯、工作压力等因素进行综合分析判断。如果确实是因为肾功能相关的问题导致的症状，医生可能会推荐相应的治疗方案；如果是其他原因，则需要针对性地进行处理。此外，保持良好的生活习惯，如适量运动、均衡饮食以及充足的休息也是非常重要的。切勿自行诊断或用药以免延误病情。如有任何疑问，请及时就医咨询专业医师意见。### 注意事项

- 不要盲目相信非专业人士的意见。
- 避免未经证实的信息误导自己做出错误决定。
- 积极配合医生指导完成相应检查



请输入您的问题（可输入序号或自定义问题）：
 3


你是一个专业的医疗助手。请根据患者的问题提供专业、准确的回答。

### 问题：
经常口干口苦，是身体哪里出了问题？

### 回答：
口干和口苦可能是多种原因导致的，具体需要结合个人的具体情况来判断可能的原因，并在必要时进行相关检查以明确诊断。以下是一些可能导致口干或口苦的情况：

1. **口腔卫生不良**：如果口腔清洁不够彻底，可能会导致细菌滋生，进而引起口臭、口苦等症状。
2. **脱水**：喝水不足也是常见的口干原因之一，保持充足的水分摄入对维持正常的生理功能至关重要。
3. **药物副作用**：一些药物（如抗抑郁药、降压药等）可能会有口干作为副作用之一。
4. **糖尿病**：高血糖水平会导致多尿，从而增加饮水量，间接影响唾液分泌，造成口干。
5. **肝胆疾病**：肝脏或胆囊疾病有时也会表现为口苦的症状。
6. **睡眠呼吸暂停综合症**：夜间频繁醒来或打鼾严重的人也可能出现早晨起床后口干的现象。
7. **干燥综合征（自身免疫性疾病的一种表现形式**：这类疾病会影响腺体的功能，导致包括唾液腺在内的多个腺体分泌减少，从而引发口干症状。

如果您的这些症状持续存在且影响日常生活，建议您尽快就医，进行全面的身体



请输入您的问题（可输入序号或自定义问题）：
 exit


感谢您的咨询，再见！
